## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place (or any two points in the Helsinki Region..).

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points 
from geopandas.tools import geocode
from pyproj import CRS

In [2]:
# read in input files

fp = r"activity_locations.txt"
data = pd.read_csv(fp, sep=";")
print(data.head())

sc_fp = "shopping_centers.shp"
shoppingcentres = gpd.read_file(sc_fp)
shoppingcentres.head(7)

     id                                       addr
0  1000    Kalevankatu 9a, 00100 Helsinki, Finland
1  1001  Yliopistonkatu 4, 00100 Helsinki, Finland


,address,id,name,addr,geometry
0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",1000,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109)
1,"Salaattiasema, 14-20, Mannerheimintie, Keskust...",1001,Forum,"Mannerheimintie 14?20, 00100 Helsinki, Finland",POINT (25496573.542 6672878.360)
2,"Captain Corvus, 11, Piispansilta, Matinkylän k...",1002,Iso-omena,"Piispansilta 11, 02230 Espoo, Finland",POINT (25485456.867 6671954.184)
3,"Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...",1003,Sello,"Leppävaarankatu 3-9, 02600 Espoo, Finland",POINT (25489491.076 6678322.265)
4,"Stockmann, 3, Vantaanportinkatu, Vantaanportti...",1004,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa, Finland",POINT (25497943.932 6686656.982)
5,"Silta, 5, Hermannin rantatie, Verkkosaari, Kal...",1005,Redi,"Hermannin rantatie 5, 00580 Helsinki, Finland",POINT (25498837.156 6674909.983)
6,None,1006,Mall of Tripla,"Fredikanterassi 1, 00520 Helsinki, Finland",POINT (25496131.965 6676190.570)


In [3]:
# Geocode activity locations

geo = geocode(data['addr'], provider='nominatim', user_agent='Gregg_Autogis', timeout=4)
geo.head()

,geometry,address
0,POINT (24.93369 60.16583),"Kalevankatu, Kamppi, Eteläinen suurpiiri, Hels..."
1,POINT (24.94874 60.16972),"UniSport, Kluuvi, 4, Yliopistonkatu, Kaisaniem..."


In [8]:
# Check coordinate reference system information (re-project/define if needed)

geo = geo.to_crs(epsg=3879)
geo.crs == shoppingcentres.crs

True

In [18]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:

def get_nearest_values(row, other_gdf, point_column='geometry', value_column="name"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["geometry"].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf["geometry"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

geo["nearest_loc"] = geo.apply(get_nearest_values, other_gdf=shoppingcentres, point_column='geometry', value_column='name', axis=1)

print("Shopping center closest to home is:", geo.loc[0,'nearest_loc'])
print("Shopping center closest to work is:", geo.loc[1,'nearest_loc'])
geo.head(7)

Shopping center closest to home is: Forum
Shopping center closest to work is: Forum


,geometry,address,nearest_loc
0,POINT (25496318.708 6672550.434),"Kalevankatu, Kamppi, Eteläinen suurpiiri, Hels...",Forum
1,POINT (25497154.366 6672982.716),"UniSport, Kluuvi, 4, Yliopistonkatu, Kaisaniem...",Forum


Can you think of other application cases for the nearest neighbour analysis?

YOUR ANSWER HERE